# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [23]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [24]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [25]:
%%capture --no-display

import pandas as pd
import hvplot.pandas

# Sample DataFrame
data = {
    "City": ["New York", "Los Angeles", "Chicago"],
    "Lng": [-74.006, -118.2437, -87.6298],
    "Lat": [40.7128, 34.0522, 41.8781],
    "Humidity": [60, 70, 55]
}
df = pd.DataFrame(data)

# Create scatter plot with geographical points using hvplot
scatter_plot = df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True,  # Enable geographical plot
    size="Humidity",  # Size based on Humidity
    scale=1,
    color="City",  # Color based on City
    alpha=0.5,
    tiles="OSM",  # Use OpenStreetMap tiles
    frame_width=700,
    frame_height=500
)

# Display the scatter plot
scatter_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# Narrow down cities that fit criteria and drop any results with null values

Vacation_df = city_data_df.loc[(city_data_df['Lat'] < 0) & (city_data_df['Wind Speed'] < 5) & 
                               (city_data_df['Max Temp'] > 0) & (city_data_df['Max Temp'] < 10), :]

# Drop any rows with null values
# YOUR CODE HERE
Vacation_df = Vacation_df.dropna()

# Display sample data
# YOUR CODE HERE
Vacation_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,ushuaia,-54.8000,-68.3000,8.81,81,75,3.09,AR,1666108231
15,15,mataura,-46.1927,168.8643,2.89,96,97,1.36,NZ,1666108238
150,150,kaeo,-35.1000,173.7833,7.96,94,62,2.17,NZ,1666108325
167,167,kaitangata,-46.2817,169.8464,6.48,77,100,4.20,NZ,1666108336
281,281,tuatapere,-46.1333,167.6833,4.20,92,79,2.40,NZ,1666108401


### Step 3: Create a new DataFrame called `hotel_df`.

In [27]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = Vacation_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ""

# Display sample data
# YOUR CODE HERE
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
5,ushuaia,AR,-54.8000,-68.3000,81,
15,mataura,NZ,-46.1927,168.8643,96,
150,kaeo,NZ,-35.1000,173.7833,94,
167,kaitangata,NZ,-46.2817,169.8464,77,
281,tuatapere,NZ,-46.1333,167.6833,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters irato search for a hotel
radius = 10000
params = {
    'radius': radius,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lat = row['Lat']
    long = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lat},{long},{radius}'
    params["bias"] = f'proximity:{lat},{long}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ushuaia - nearest hotel: No hotel found
mataura - nearest hotel: No hotel found
kaeo - nearest hotel: No hotel found
kaitangata - nearest hotel: No hotel found
tuatapere - nearest hotel: No hotel found
dunedin - nearest hotel: No hotel found
christchurch - nearest hotel: No hotel found
waipawa - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
5,ushuaia,AR,-54.8000,-68.3000,81,No hotel found
15,mataura,NZ,-46.1927,168.8643,96,No hotel found
150,kaeo,NZ,-35.1000,173.7833,94,No hotel found
167,kaitangata,NZ,-46.2817,169.8464,77,No hotel found
281,tuatapere,NZ,-46.1333,167.6833,92,No hotel found
336,dunedin,NZ,-45.8742,170.5036,73,No hotel found
372,christchurch,NZ,-43.5333,172.6333,95,No hotel found
528,waipawa,NZ,-41.4122,175.5153,91,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [30]:

%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE


import pandas as pd
import hvplot.pandas

# Sample data for demonstration
data = {
    'City': ['City1', 'City2', 'City3'],
    'Lat': [40.7128, 34.0522, 37.7749],
    'Lng': [-74.0060, -118.2437, -122.4194]
}

# Create a sample hotel_df DataFrame
hotel_df = pd.DataFrame(data)

# Configure the map plot
map_plot_1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]